In [68]:
import requests
import json
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
'''Data Collection:
We will first collect our skincare product data from amzon api through keywords like "moisturizer", "sunscreen", 
"cleanser", "serum", "exfoliator", "lipbalm", "clay mask", "face mask", "sheet mask", "cleansing oil", "cleansing balm",
"essence", "eye cream", "eye mask", "face scrub", and "body scrub".

For this, we will generate a list of keywords and iterate the api request link for the first 100 pages on amazon.com
through these keywords to get the resultant output files in .json format. The output files relevant to each category
will be available on the local computer.
'''

# Defining the base URL
base_url = "https://amazon-price1.p.rapidapi.com/search"

# Common headers
headers = {
    "X-RapidAPI-Key": "23ac224d0cmshdf63ce432969608p19db29jsn0851cfb9e96b",
    "X-RapidAPI-Host": "amazon-price1.p.rapidapi.com"
}

# Defining a list of product categories
product_categories = ["moisturizer", "sunscreen", "cleanser", "serum", "exfoliator", "lipbalm", "clay mask", "face mask", "sheet mask", "cleansing oil", "cleansing balm", "essence", "eye cream", "eye mask", "face scrub", "body scrub"]

# Defining the maximum number of pages (e.g., 15)
max_pages = 100

# Looping through each product category and send a request for each page
for category in product_categories:
    # Initializing an empty list to accumulate data for all pages
    all_pages_data = []
    
    for page in range(1, max_pages + 1):
        querystring = {"keywords": category, "marketplace": "US", "page": page}
        response = requests.get(base_url, headers=headers, params=querystring)
        
        if response.status_code == 200:
            data = response.json()
            # Append the data for the current page to the list
            all_pages_data.extend(data)
        else:
            print(f"Error for {category}, Page {page}: {response.status_code}")
    
    # Saving all the data for the current category to a JSON file
    filename = f"{category}.json" 
    with open(filename, 'w') as json_file:
        json.dump(all_pages_data, json_file, indent=4)
    
    print(f"Data for {category} saved to {filename}")


Data for moisturizer saved to moisturizer.json
Data for sunscreen saved to sunscreen.json
Data for cleanser saved to cleanser.json
Data for serum saved to serum.json
Data for exfoliator saved to exfoliator.json
Data for lipbalm saved to lipbalm.json
Data for clay mask saved to clay mask.json
Data for face mask saved to face mask.json
Data for sheet mask saved to sheet mask.json
Data for cleansing oil saved to cleansing oil.json
Data for cleansing balm saved to cleansing balm.json
Data for essence saved to essence.json
Data for eye cream saved to eye cream.json
Data for eye mask saved to eye mask.json
Data for face scrub saved to face scrub.json
Data for body scrub saved to body scrub.json


In [66]:
'''
The resultant data were 16 .json files pertaining to each keyword. Each .json file consisted of the first 100 pages
worth results of the keyword on amazon. These data files were concatenated and then stored into a final dataframe 
called final_df. This final_df was subsequently converted to a downloadable excel file.
'''
import pandas as pd
import json

# Defining a list of product categories
product_categories = ["moisturizer", "sunscreen", "cleanser", "serum", "exfoliator", "lipbalm", "clay mask", "face mask", "sheet mask", "cleansing oil", "cleansing balm", "essence", "eye cream", "eye mask", "face scrub", "body scrub"]

# Creating an empty DataFrame to store the data
final_df = pd.DataFrame()

# Looping through each product category
for category in product_categories:
    # Loading data from the JSON file for the current category
    filename = f"{category}.json"
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    
    # Createing a DataFrame from the JSON data
    df = pd.DataFrame(data)
    
    # Adding a new column for the product category
    df['Category'] = category
    
    # Concatenating the DataFrame for the current category to the final DataFrame
    final_df = pd.concat([final_df, df], axis=0, ignore_index=True)

In [69]:
# Saving the final DataFrame as an Excel file
excel_file = 'amazon_data2.xlsx'
final_df.to_excel(excel_file, index=False)

print(f'Data saved to {excel_file}')

PermissionError: [Errno 13] Permission denied: 'amazon_data2.xlsx'

In [73]:
'''
Some of the product_names in the dataframe were random (e.g. Okay?) and led to nowhere. 
So all the rows corresponding to such titles were removed.
'''

product_df=pd.read_excel("C:\\Users\\vakit\\amazon_data2.xlsx")

# Renaming specific columns
product_df = product_df.rename(columns={'title': 'product_name', 'subtitle': 'product_company'})

# Removing rows where 'title' equals 'Okay?'
product_df = product_df[df['title'] != 'Okay?']
product_df

C:\Users\vakit\AppData\Local\Temp\ipykernel_328\773230460.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  product_df = product_df[df['title'] != 'Okay?']


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [74]:
'''A closer inspection shows that this dataset also extracted a lot of makeup products like lipstick, lipgloss, 
mascara, concealer, foundation , eyeliner, fixing powder, concealer, lipgloss, eye tint, primer, eye shadow, etc. 
Our focus is only on skincare products so these makeup products were removed.'''

# Listing keywords to search for
excluding_keywords = [
    'lipstick', 'lipgloss', 'mascara', 'concealer', 'foundation',
    'eyeliner', 'fixing powder', 'concealer', 'lipgloss', 'eye tint',
    'primer', 'eye shadow'
]

# Filtering rows where 'product_name' contains any of the excluding keywords
removal_rows = product_df['product_name'].str.contains('|'.join(excluding_keywords), case=False, regex=True)

In [71]:
# Drop rows based on the removal_rows
product_df = product_df[~removal_rows]

# Reset the index if needed
product_df.reset_index(drop=True, inplace=True)

product_df

C:\Users\vakit\AppData\Local\Temp\ipykernel_328\3106833020.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  product_df = product_df[~removal_rows]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Load your Excel file with product information into a DataFrame
excel_file = "C:\\Users\\vakit\\amazon_data2.xlsx"
df = pd.read_excel(excel_file)

# Initialize lists to store scraped data
ingredient_list = []
highlights_list = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Construct the search query URL for Incidecoder using the product name
    product_name = row['title']  # Adjust column name as needed
    search_url = f"https://incidecoder.com/search?query={product_name}"
    
    # Send an HTTP GET request to the Incidecoder search page
    response = requests.get(search_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extract ingredient and highlights information
        ingredients = soup.find("div", class_="table-wrapper").find_all("td", class_="td-left")
        highlights = soup.find("div", class_="table-wrapper").find_all("td", class_="td-right")
        
        if ingredients and highlights:
            # Extract the text from each element and join them into a single string
            ingredient_text = ", ".join([ingredient.get_text(strip=True) for ingredient in ingredients])
            highlights_text = ", ".join([highlight.get_text(strip=True) for highlight in highlights])
            
            # Append the scraped data to the lists
            ingredient_list.append(ingredient_text)
            highlights_list.append(highlights_text)
        else:
            ingredient_list.append("N/A")  # Handle cases where data is not found
            highlights_list.append("N/A")
    else:
        ingredient_list.append("N/A")
        highlights_list.append("N/A")

# Add the scraped data as new columns in the DataFrame
df['Ingredients'] = ingredient_list
df['Highlights'] = highlights_list

# Save the updated DataFrame to a new Excel file
output_excel_file = "products_with_info.xlsx"
df.to_excel(output_excel_file, index=False)

print(f"Data saved to {product_ingredient_excel_file}")


AttributeError: 'NoneType' object has no attribute 'find_all'